In [1]:
from line_solver import *
import numpy as np
GlobalConstants.set_verbose(VerboseLevel.STD)

In [2]:
# Fork-Join Example 11
model = Network('model')

In [3]:
# Create network
delay = Delay(model, 'Delay1')
delay2 = Delay(model, 'Delay2')
queue1 = Queue(model,'Queue1',SchedStrategy.FCFS)
queue2 = Queue(model,'Queue2',SchedStrategy.FCFS)
fork = Fork(model,'Fork')
join = Join(model,'Join', fork)

jobclass1 = ClosedClass(model, 'class1', 10, delay, 0)

In [4]:
# Service configurations
queue1.set_service(jobclass1, Exp(1.0))
queue2.set_service(jobclass1, Exp(1.0))
delay.set_service(jobclass1, Exp(0.5))
delay2.set_service(jobclass1, Exp(2.0))

In [5]:
# Set routing matrix
P = model.init_routing_matrix()

# Serial routing through delays: delay -> delay2 -> fork
P.set(jobclass1, jobclass1, delay, delay2, 1.0)
P.set(jobclass1, jobclass1, delay2, fork, 1.0)
P.set(jobclass1, jobclass1, fork, queue1, 1.0)
P.set(jobclass1, jobclass1, fork, queue2, 1.0)
P.set(jobclass1, jobclass1, queue1, join, 1.0)
P.set(jobclass1, jobclass1, queue2, join, 1.0)
P.set(jobclass1, jobclass1, join, delay, 1.0)

model.link(P)

In [6]:
# Solve with multiple methods
solvers = [JMT(model, seed=23000), MVA(model)]
for i, solver in enumerate(solvers):
    print(f'SOLVER {i+1}: {solver.get_name()}')
    avgTable = solver.avg_table()

SOLVER 1: JMT


Station JobClass   QLen   Util  RespT  ResidT   ArvR   Tput
 Delay1   class1 1.8303 1.8303 2.0032  2.0032 0.9371 0.9371
 Delay2   class1 0.4694 0.4694 0.5048  0.5048 0.9306 0.9306
 Queue1   class1 5.7380 0.9401 6.1485  6.1485 0.9314 0.9314
 Queue2   class1 5.3036 0.9261 5.6327  5.6327 0.9381 0.9381
   Join   class1 4.3128 0.9465 2.3605  2.3605 0.9312 0.9312
SOLVER 2: MVA
Station JobClass  QLen   Util  RespT  ResidT   ArvR   Tput
 Delay1   class1 2.101 2.1010  2.500   2.500 0.8404 0.8404
 Delay2   class1 2.101 2.1010  2.500   2.500 0.8404 0.8404
 Queue1   class1 5.266 0.8404  6.266   6.266 0.8404 0.8404
 Queue2   class1 5.266 0.8404  6.266   6.266 0.8404 0.8404
   Join   class1 5.266 0.0000  3.133   3.133 0.8404 0.8404
